In [1]:
import os
import glob

from pathlib import Path
import shutil

import tensorflow as tf
import tf_keras as keras
import numpy as np
from tqdm import tqdm
import yaml
import ray

from rl.network import ResNet
from rl.mcts import MCTS
from rl.buffer import ReplayBuffer, Sample
from rl.game import Game, encode_state

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
index = "20241130"

base_path = "graphs"
index = "20241201"
qubits = config["game_settings"]["N"]
training_settings = config["training_settings"]
network_settings = config["network_settings"]
mcts_settings = config["mcts_settings"]
num_cpus = training_settings["num_cpus"]
num_gpus = training_settings["num_gpus"]
n_episodes = training_settings["n_episodes"]
buffer_size = training_settings["buffer_size"]
batch_size = training_settings["batch_size"]
epochs_per_update = training_settings["epochs_per_update"]
update_period = training_settings["update_period"]
save_period = training_settings["save_period"]
eval_period = training_settings.get("eval_period", 100)

@ray.remote(num_cpus=num_cpus, num_gpus=num_gpus)
def selfplay(weights, qubits, current_episode, config):
    record = []
    game = Game(qubits, config)
    state = game.get_initial_state()
    game.reset_used_columns()
    network = ResNet(action_space=len(game.coupling_map), config=config)
    network.predict(encode_state(state, qubits))
    network.set_weights(weights)

    mcts = MCTS(qubits=qubits, network=network, config=config)
    done = False
    total_score = 0
    step_count = 0
    prev_action = None

    while not done and step_count < game.MAX_STEPS:
        mcts_policy = mcts.search(
            root_state=state,
            prev_action=prev_action,
            num_simulations=mcts_settings["num_mcts_simulations"],
        )
        if prev_action is not None:
            indices = [i for i in range(len(game.coupling_map)) if i != prev_action]
            valid_actions = game.get_valid_actions(state, prev_action)
            prob = mcts_policy[valid_actions]
            prob = prob / prob.sum()
            action = np.random.choice(valid_actions, p=prob)
        else:
            indices = list(range(len(game.coupling_map)))
            prob = mcts_policy
            action = np.random.choice(indices, p=prob)
        record.append(Sample(state.copy(), mcts_policy, reward=None))
        state, done, action_score = game.step(state, action, prev_action)
        prev_action = action
        # print(state, action_score, done)
        total_score += action_score
        step_count += 1

    reward = game.get_reward(state, total_score)
    for sample in record:
        sample.reward = reward
    return record


def evaluate_self_play(qubits, network, config):
    pattern = os.path.join(base_path, f"adj_matrix_{qubits}_*.npy")
    file_paths = glob.glob(pattern)
    avg_depth = []
    avg_counts = []
    for file_path in tqdm(file_paths):
        state = np.load(file_path)
        game = Game(qubits, config)
        swap_pairs = []
        done = False
        step_count = 0
        prev_action = None
        while not done and step_count < game.MAX_STEPS:
            encoded_state = encode_state(state, qubits)
            input_state = np.expand_dims(encoded_state, axis=0)
            policy_output, value_output = network.predict(input_state)
            policy = np.array(policy_output)[0]
            # policy = tf.nn.softmax(policy_logits).numpy()[0]
            # valid_actions = game.get_valid_actions(state, prev_action)
            if prev_action is not None:
                indices = [i for i in range(len(game.coupling_map)) if i != prev_action]
                try:
                    valid_actions = game.get_valid_actions(state, prev_action)
                    prob = policy[valid_actions]
                except:
                    prob = policy[indices]
                action = np.random.choice(valid_actions, p=prob / prob.sum())
            else:
                indices = list(range(len(game.coupling_map)))
                prob = policy
                action = np.random.choice(indices, p=prob / prob.sum())
            selected_action = game.coupling_map[action]
            swap_pairs.append(selected_action)
            state, done, _ = game.step(state, action, prev_action)
            prev_action = action
            step_count += 1
        if not done:
            depth = game.MAX_STEPS
            swap_count = game.MAX_STEPS
        else:
            depth = game.current_layer
            swap_count = len(swap_pairs)
        avg_counts.append(swap_count)
        avg_depth.append(depth)
    return avg_depth, avg_counts

2024-11-30 22:52:45.635252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 22:52:45.647885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 22:52:45.651538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 22:52:45.665763: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 22:52:46.572445: W tensorflow/compiler/tf2

In [ ]:
ray.init(num_cpus=num_cpus, num_gpus=num_gpus, local_mode=False, include_dashboard=True)
print(ray.available_resources())
logdir = Path("log")
if logdir.exists():
    shutil.rmtree(logdir)
summary_writer = tf.summary.create_file_writer(str(logdir))

game = Game(qubits, config)
network = ResNet(action_space=len(game.coupling_map), config=config)

dummy_state = encode_state(game.state, qubits)
network.predict(encode_state(game.state, qubits))
current_weights = ray.put(network.get_weights())

optimizer = keras.optimizers.legacy.Adam(
    learning_rate=network_settings["learning_rate"]
)

replay = ReplayBuffer(buffer_size=buffer_size)
work_in_progresses = [selfplay.remote(current_weights, qubits, i, config) for i in range(num_gpus - 1)]
n_updates = 0

n = 0
while n < n_episodes:
    for _ in tqdm(range(update_period)):
        # finished = selfplay(current_weights, qubits, n, config)
        # replay.add_record(finished)
        finished, work_in_progresses = ray.wait(work_in_progresses, num_returns=1)
        replay.add_record(ray.get(finished[0]))
        work_in_progresses.extend([selfplay.remote(current_weights, qubits, True)])
        n += 1
    print("-" * 50)
    if len(replay) >= batch_size:
        num_iters = epochs_per_update * (len(replay) // batch_size)
        value_loss_weight = 1
        policy_loss_weight = 1
        for i in tqdm(range(num_iters)):
            states, mcts_policy, rewards = replay.get_minibatch(batch_size=batch_size)
            with tf.GradientTape() as tape:
                p_pred, v_pred = network(states, training=True)
                value_loss = tf.square(rewards - v_pred)
                policy_loss = -tf.reduce_sum(
                    mcts_policy * tf.math.log(p_pred + 1e-5), axis=1, keepdims=True
                )
                loss = tf.reduce_mean(
                    value_loss_weight * value_loss + policy_loss_weight * policy_loss
                )
            grads = tape.gradient(loss, network.trainable_variables)
            grads, _ = tf.clip_by_global_norm(grads, 1.0)
            optimizer.apply_gradients(zip(grads, network.trainable_variables))
            n_updates += 1

            if i % 10 == 0:
                with summary_writer.as_default():
                    tf.summary.scalar(
                        "value_loss", tf.reduce_mean(value_loss), step=n_updates
                    )
                    tf.summary.scalar(
                        "policy_loss", tf.reduce_mean(policy_loss), step=n_updates
                    )

        current_weights = ray.put(network.get_weights())

    if n % save_period == 0:
        network.save(f"checkpoints/network{qubits}_{index}_{n}", save_format="tf")
        network.save_weights(f"checkpoints/network{qubits}_{index}_{n}.weights.h5")
        print("-" * 50)
    if n % eval_period == 0:
        depth, count = evaluate_self_play(qubits, network, config)
        print(
            f"Episode {n}: SWAP depth is {np.mean(depth)}, SWAP count is {np.mean(count)}"
        )
        print("-" * 50)

100%|██████████| 50/50 [06:06<00:00,  7.33s/it]
W0000 00:00:1732964513.030224   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.031754   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.033222   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.034482   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.035755   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.037339   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.039057   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732964513.040184   12529 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be 

--------------------------------------------------


100%|██████████| 50/50 [08:04<00:00,  9.69s/it]


--------------------------------------------------


100%|██████████| 50/50 [07:35<00:00,  9.10s/it]


...

100%|██████████| 50/50 [05:35<00:00,  6.72s/it]


--------------------------------------------------


100%|██████████| 50/50 [05:54<00:00,  7.08s/it]


--------------------------------------------------


100%|██████████| 50/50 [05:01<00:00,  6.03s/it]


In [ ]:
game = Game(qubits, config)
network = ResNet(action_space=len(game.coupling_map), config=config)
network = keras.models.load_model(f"checkpoints/network{qubits}_{index}_700")

In [ ]:
for _ in range(20):
    game = Game(qubits, config)
    state = game.state
    ans = []
    done = False
    total_score = 0
    step_count = 0
    prev_action = None
    print(state)
    while not done and step_count < game.MAX_STEPS:
        encoded_state = encode_state(state, qubits)
        input_state = np.expand_dims(encoded_state, axis=0)

        policy_output, value_output = network.predict(input_state)
        policy = policy_output[0]
        if prev_action is not None:
            indices = [i for i in range(len(game.coupling_map)) if i != prev_action]
            prob = policy[indices]
            action = np.random.choice(indices, p=prob / prob.sum())
        else:
            indices = list(range(len(game.coupling_map)))
            action = np.random.choice(indices, p=policy)
        selected_action = game.coupling_map[action]
        ans.append(selected_action)
        state, done, _ = game.step(state, action, prev_action)
        prev_action = action
        step_count += 1
    if done:
        print(f"Game finished successfully in {step_count} steps with {ans}")
    else:
        print(f"Game terminated after reaching the maximum steps ({game.MAX_STEPS}).")
        print(f"Total score: {total_score}")

In [12]:
depths = []
for _ in range(10):
    depth, count = evaluate_self_play(qubits, network, config)
    depths.append(depth)
min_depth = np.min(np.vstack(depths), axis=0)

1/1 [==============================] - 0s 31ms/step


 30%|███       | 9/30 [00:11<00:29,  1.42s/it]

1/1 [==============================] - 0s 19ms/step


 33%|███▎      | 10/30 [00:12<00:25,  1.27s/it]

1/1 [==============================] - 0s 20ms/step


...

 93%|█████████▎| 28/30 [00:30<00:01,  1.07it/s]

1/1 [==============================] - 0s 18ms/step


 97%|█████████▋| 29/30 [00:31<00:00,  1.24it/s]

1/1 [==============================] - 0s 16ms/step


100%|██████████| 30/30 [00:31<00:00,  1.06s/it]


In [13]:
min_depth

array([1, 5, 6, 7, 6, 7, 7, 1, 7, 3, 2, 5, 3, 5, 7, 7, 8, 4, 6, 1, 7, 8,
       4, 6, 4, 5, 4, 2, 2, 2])

In [14]:
np.mean(min_depth)

4.733333333333333

array([1, 5, 6, 4, 6, 7, 7, 2, 6, 3, 2, 5, 3, 7, 5, 7, 6, 3, 5, 2, 4, 9,
       2, 6, 5, 5, 3, 2, 2, 1]) -> 4.366666666666666